In [1]:
!pip install catboost

     |████████████████████████████████| 10.9MB 2.3MB/s eta 0:00:01


In [33]:
import numpy as np
import pandas as pd
import catboost
from sklearn.model_selection import train_test_split
import ast
import scipy


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
from itertools import product, chain
from tqdm import tqdm

In [2]:
train = pd.read_csv("train_jqd04QH.csv")

In [3]:
def experience_cat(x):
    if pd.isna(x):
        return x
    elif str(x) == "<1":
        return "Less than 1"
    elif str(x) == ">20":
        return "Greater than 20"
    elif isinstance(ast.literal_eval(str(x)) , int):
        if int(x) <= 5 and int(x) >= 1:
            return "Between 1 and 5"
        elif int(x) <= 10 and int(x) > 5:
            return "Between 6 and 10"
        elif int(x) <= 15 and int(x) > 10:
            return "Between 11 and 15"
        elif int(x) <= 20 and int(x) > 15:
            return "Between 16 and 20"
    else:
        return x

In [4]:
train["enrollee_id"] = train["enrollee_id"].astype(str)
train["target"] = train.target.astype(str)
train["experience_categorical"] = train["experience"].apply(lambda x : experience_cat(x))

In [5]:
train["city"] = train["city"].astype("str")
train["gender"] = train["gender"].astype("str")
train["enrolled_university"] = train["enrolled_university"].astype("str")
train["education_level"] = train["education_level"].astype("str")
train["major_discipline"] = train["major_discipline"].astype("str")
train["experience_categorical"] = train["experience_categorical"].astype("str")
train["company_size"] = train["company_size"].astype("str")
train["company_type"] = train["company_type"].astype("str")
train["last_new_job"] = train["last_new_job"].astype("str")
train["experience"] = train["experience"].astype("str")
train["relevent_experience"] = train["relevent_experience"].astype("str")

In [6]:
train.columns

Index(['enrollee_id', 'city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours', 'target', 'experience_categorical'],
      dtype='object')

In [7]:
cols = ['city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours' , 'experience_categorical']

cat_cols = ['city',  'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'experience_categorical']

cat_cols_index = [c for c, col in enumerate(cols) if col in cat_cols]

In [8]:
train[cat_cols].dtypes

city                      object
gender                    object
relevent_experience       object
enrolled_university       object
education_level           object
major_discipline          object
experience                object
company_size              object
company_type              object
last_new_job              object
experience_categorical    object
dtype: object

In [9]:
cat_cols_index

[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12]

In [27]:
## get the labels
y = train.target.values

x = train[cols].values

In [11]:
#Test data 
test = pd.read_csv("test_KaymcHn.csv")

In [12]:
test["enrollee_id"] = test["enrollee_id"].astype(str)
test["experience_categorical"] = test["experience"].apply(lambda x : experience_cat(x))

test["city"] = test["city"].astype("str")
test["gender"] = test["gender"].astype("str")
test["enrolled_university"] = test["enrolled_university"].astype("str")
test["education_level"] = test["education_level"].astype("str")
test["major_discipline"] = test["major_discipline"].astype("str")
test["experience_categorical"] = test["experience_categorical"].astype("str")
test["company_size"] = test["company_size"].astype("str")
test["company_type"] = test["company_type"].astype("str")
test["last_new_job"] = test["last_new_job"].astype("str")
test["experience"] = test["experience"].astype("str")
test["relevent_experience"] = test["relevent_experience"].astype("str")

In [28]:
X_test = test[cols].values

In [14]:
#
# Create training and validation sets
#
#x, x_test, y, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)
#train_dataset = catboost.Pool(data=x,label=y,cat_features=cat_cols)
#eval_dataset = catboost.Pool(data=x_test,label=y_test,cat_features=cat_cols)

#

In [34]:
#Reference : https://www.kaggle.com/miklgr500/catboost-with-gridsearch-cv

RANDOM_STATE = 0
  
def cross_val(X, y, X_test, param, cat_features, n_splits=3):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    
    acc = []
    predict = None
    
    for tr_ind, val_ind in skf.split(X, y):
        X_train = X[tr_ind]
        y_train = y[tr_ind]
        
        X_valid = X[val_ind]
        y_valid = y[val_ind]
        
        clf = CatBoostClassifier(iterations=param["iterations"],
                                loss_function = param['loss_function'],
                                depth=param['depth'],
                                l2_leaf_reg = param['l2_leaf_reg'],
                                eval_metric = 'AUC',
                                leaf_estimation_iterations = 10,
                                use_best_model=True,
                                logging_level='Silent'
        )
        
        clf.fit(X_train, 
                y_train,
                cat_features=cat_features,
                eval_set=(X_valid, y_valid)
        )
        
        y_pred = clf.predict(X_valid)
        accuracy = roc_auc_score(y_valid, y_pred)
        acc.append(accuracy)
    return sum(acc)/n_splits
    
def catboost_GridSearchCV(X, y, X_test, params, cat_features, n_splits=5):
    ps = {'acc':0,
          'param': []
    }
    
    predict=None
    
    for prms in tqdm(list(ParameterGrid(params)), ascii=True, desc='Params Tuning:'):
                          
        acc = cross_val(X, y, X_test, prms, cat_features, n_splits=5)

        if acc>ps['acc']:
            ps['acc'] = acc
            ps['param'] = prms
    print('Acc: '+str(ps['acc']))
    print('Params: '+str(ps['param']))
    
    return ps['param']
    
    


In [35]:
params = {'depth':[2, 3, 4],
          'loss_function': ['Logloss', 'CrossEntropy'],
          'l2_leaf_reg':np.logspace(-20, -19, 3) , 'iterations': [500, 1000]
}
    
param = catboost_GridSearchCV(x, y, X_test, params, cat_cols_index)

clf = CatBoostClassifier(iterations=param["iterations"],
                        loss_function = param['loss_function'],
                        depth=param['depth'],
                        l2_leaf_reg = param['l2_leaf_reg'],
                        eval_metric = 'AUC',
                        leaf_estimation_iterations = 10,
                        use_best_model=True
)
X_train, X_valid, y_train, y_valid = train_test_split(x,
                                                    y, 
                                                    shuffle=True,
                                                    random_state=RANDOM_STATE,
                                                    train_size=0.8,
                                                    stratify=y
)



Params Tuning:: 100%|##########| 36/36 [31:58<00:00, 89.37s/it]

Acc: 0.5003809936630448
Params: {'depth': 4, 'iterations': 500, 'l2_leaf_reg': 1e-20, 'loss_function': 'Logloss'}


NameError: name 'cat_features' is not defined

In [36]:
clf.fit(X_train, 
        y_train,
        cat_features=cat_cols_index,
        logging_level='Silent',
        eval_set=(X_valid, y_valid)
)

In [53]:
clf.feature_importances_

array([ 9.38043764,  9.87784093,  4.27448384,  2.46499027,  8.967665  ,
        5.61261731,  8.2052408 ,  9.89509084, 13.49112594,  6.07878117,
        8.42212836,  4.90463223,  8.42496567])

In [60]:
clf.get_best_iteration

<bound method _CatBoostBase.get_best_iteration of <catboost.core.CatBoostClassifier object at 0x1a27236c50>>

In [38]:
import pickle
sfile = open('model_catboost.pkl', 'wb')
pickle.dump(clf, sfile)

In [41]:
pred = clf.predict_proba(X_test)

In [46]:
pred1 = pd.DataFrame(pred)

In [49]:
sub = pd.DataFrame({'enrollee_id':test['enrollee_id'].tolist()})
sub["target"] = pred1[1]
sub.to_csv('Submission_CatBoost.csv',index=False)

## Submission 7

In [63]:
clf_new = CatBoostClassifier(iterations=500 , depth=4 , l2_leaf_reg=1e-20 , loss_function="Logloss" , eval_metric = 'AUC',
                        leaf_estimation_iterations = 10,
                        use_best_model=False)

In [64]:
clf_new.fit(x, 
        y,
        cat_features=cat_cols_index)

0:	total: 12.3ms	remaining: 6.12s
1:	total: 22.4ms	remaining: 5.57s
2:	total: 28.5ms	remaining: 4.71s
3:	total: 33.7ms	remaining: 4.18s
4:	total: 40.3ms	remaining: 3.99s
5:	total: 46.3ms	remaining: 3.81s
6:	total: 56.2ms	remaining: 3.96s
7:	total: 67.4ms	remaining: 4.14s
8:	total: 74.7ms	remaining: 4.07s
9:	total: 81.1ms	remaining: 3.98s
10:	total: 91ms	remaining: 4.05s
11:	total: 97.9ms	remaining: 3.98s
12:	total: 109ms	remaining: 4.09s
13:	total: 120ms	remaining: 4.16s
14:	total: 130ms	remaining: 4.2s
15:	total: 140ms	remaining: 4.24s
16:	total: 150ms	remaining: 4.26s
17:	total: 160ms	remaining: 4.28s
18:	total: 165ms	remaining: 4.17s
19:	total: 175ms	remaining: 4.21s
20:	total: 181ms	remaining: 4.13s
21:	total: 186ms	remaining: 4.04s
22:	total: 192ms	remaining: 3.98s
23:	total: 201ms	remaining: 3.99s
24:	total: 211ms	remaining: 4.01s
25:	total: 224ms	remaining: 4.08s
26:	total: 232ms	remaining: 4.07s
27:	total: 242ms	remaining: 4.08s
28:	total: 247ms	remaining: 4.01s
29:	total: 258m

243:	total: 2.66s	remaining: 2.79s
244:	total: 2.68s	remaining: 2.79s
245:	total: 2.69s	remaining: 2.78s
246:	total: 2.7s	remaining: 2.77s
247:	total: 2.71s	remaining: 2.75s
248:	total: 2.72s	remaining: 2.74s
249:	total: 2.73s	remaining: 2.73s
250:	total: 2.74s	remaining: 2.72s
251:	total: 2.75s	remaining: 2.71s
252:	total: 2.76s	remaining: 2.69s
253:	total: 2.77s	remaining: 2.68s
254:	total: 2.78s	remaining: 2.67s
255:	total: 2.8s	remaining: 2.67s
256:	total: 2.81s	remaining: 2.66s
257:	total: 2.82s	remaining: 2.65s
258:	total: 2.83s	remaining: 2.63s
259:	total: 2.84s	remaining: 2.62s
260:	total: 2.85s	remaining: 2.61s
261:	total: 2.87s	remaining: 2.6s
262:	total: 2.88s	remaining: 2.59s
263:	total: 2.89s	remaining: 2.58s
264:	total: 2.9s	remaining: 2.57s
265:	total: 2.91s	remaining: 2.56s
266:	total: 2.93s	remaining: 2.55s
267:	total: 2.94s	remaining: 2.55s
268:	total: 2.95s	remaining: 2.54s
269:	total: 2.96s	remaining: 2.52s
270:	total: 2.98s	remaining: 2.51s
271:	total: 2.99s	remain

489:	total: 5.74s	remaining: 117ms
490:	total: 5.75s	remaining: 105ms
491:	total: 5.77s	remaining: 93.8ms
492:	total: 5.78s	remaining: 82.1ms
493:	total: 5.79s	remaining: 70.4ms
494:	total: 5.81s	remaining: 58.7ms
495:	total: 5.82s	remaining: 46.9ms
496:	total: 5.83s	remaining: 35.2ms
497:	total: 5.85s	remaining: 23.5ms
498:	total: 5.86s	remaining: 11.7ms
499:	total: 5.87s	remaining: 0us


In [ ]:
pred = clf_new.predict_proba(X_test)
pred1 = pd.DataFrame(pred)

sub = pd.DataFrame({'enrollee_id':test['enrollee_id'].tolist()})
sub["target"] = pred1[1]
sub.to_csv('Submission7_CatBoost.csv',index=False)